# 中等项目1：完整客户流失预测系统

## 项目描述

构建一个完整的客户流失预测系统，包括数据探索、特征工程、模型训练、评估和结果分析。这是一个完整的数据科学项目流程。

## 学习目标

通过本项目，你将学会：
1. 如何进行完整的数据探索（EDA）
2. 如何进行特征工程和预处理
3. 如何训练和优化模型
4. 如何全面评估模型性能
5. 如何分析特征重要性

## 项目流程

1. 数据探索（EDA）：统计分析、可视化、相关性分析
2. 数据预处理：缺失值处理、异常值处理、特征编码
3. 特征工程：特征选择、特征创建、特征变换
4. 模型训练：训练多个模型，使用交叉验证
5. 模型评估：多种评估指标、ROC曲线、混淆矩阵
6. 结果分析：特征重要性分析、业务建议


In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                            roc_auc_score, roc_curve, precision_recall_curve, auc)
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)
%matplotlib inline

print("环境准备完成！")


## 1. 数据生成与加载

生成模拟客户流失数据。在实际项目中，你需要从数据库或文件中加载真实数据。


In [ ]:
# 生成模拟客户流失数据
# 特征包括：年龄、收入、使用时长、投诉次数、服务次数、满意度等
np.random.seed(42)
n_samples = 2000

# 生成特征
data = {
    'age': np.random.normal(35, 10, n_samples).clip(18, 80),
    'income': np.random.normal(50000, 15000, n_samples).clip(20000, 150000),
    'usage_months': np.random.normal(24, 12, n_samples).clip(1, 60),
    'complaints': np.random.poisson(2, n_samples),
    'service_calls': np.random.poisson(3, n_samples),
    'satisfaction': np.random.uniform(1, 10, n_samples),
    'contract_type': np.random.choice(['Month-to-month', 'One year', 'Two year'], n_samples),
    'payment_method': np.random.choice(['Electronic', 'Mailed check', 'Bank transfer'], n_samples)
}

df = pd.DataFrame(data)

# 生成目标变量（流失）：基于特征的逻辑组合
# 流失概率与投诉次数、满意度、合同类型等相关
churn_prob = (
    0.1 * (df['complaints'] / 10) +
    0.2 * (1 - df['satisfaction'] / 10) +
    0.1 * (df['contract_type'] == 'Month-to-month').astype(int) +
    np.random.normal(0, 0.1, n_samples)
)
df['churn'] = (churn_prob > 0.3).astype(int)

print(f"数据形状: {df.shape}")
print(f"流失率: {df['churn'].mean():.2%}")
print(f"\n数据预览:")
print(df.head())
print(f"\n数据类型:")
print(df.dtypes)
print(f"\n缺失值:")
print(df.isnull().sum())


## 2. 数据探索（EDA）

探索数据的基本特征、分布和相关性。


In [ ]:
# 数据统计信息
print("数值特征统计:")
print(df.describe())

# 类别特征统计
print("\n类别特征统计:")
print(f"合同类型分布:")
print(df['contract_type'].value_counts())
print(f"\n支付方式分布:")
print(df['payment_method'].value_counts())

# 流失率分析
print(f"\n流失率分析:")
print(f"总体流失率: {df['churn'].mean():.2%}")
print(f"\n按合同类型的流失率:")
print(df.groupby('contract_type')['churn'].agg(['mean', 'count']))
print(f"\n按支付方式的流失率:")
print(df.groupby('payment_method')['churn'].agg(['mean', 'count']))


In [ ]:
# 可视化数据分布
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 数值特征分布
numeric_cols = ['age', 'income', 'usage_months', 'complaints', 'service_calls', 'satisfaction']
for i, col in enumerate(numeric_cols):
    row, col_idx = i // 3, i % 3
    axes[row, col_idx].hist(df[col], bins=30, alpha=0.7, edgecolor='black')
    axes[row, col_idx].set_title(f'{col} 分布')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('频数')
    axes[row, col_idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# 相关性分析
numeric_df = df.select_dtypes(include=[np.number])
correlation = numeric_df.corr()

# 绘制相关性热力图
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('特征相关性热力图')
plt.tight_layout()
plt.show()

# 流失与各特征的关系
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for i, col in enumerate(numeric_cols):
    row, col_idx = i // 3, i % 3
    df.boxplot(column=col, by='churn', ax=axes[row, col_idx])
    axes[row, col_idx].set_title(f'{col} vs 流失')
    axes[row, col_idx].set_xlabel('流失')
    axes[row, col_idx].set_ylabel(col)

plt.tight_layout()
plt.show()


## 3. 数据预处理

处理类别特征、标准化数值特征。


In [ ]:
# 准备特征和目标
X = df.drop('churn', axis=1)
y = df['churn']

# 处理类别特征：One-Hot编码
X_encoded = pd.get_dummies(X, columns=['contract_type', 'payment_method'], drop_first=True)

print(f"编码后特征数量: {X_encoded.shape[1]}")
print(f"特征名称: {list(X_encoded.columns)}")

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

# 标准化数值特征
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"训练集流失率: {y_train.mean():.2%}")
print(f"测试集流失率: {y_test.mean():.2%}")


## 4. 模型训练

训练逻辑回归模型，使用交叉验证评估。
